In [ ]:
# basic dependencies; setup logger
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, copy
import matplotlib.pyplot as plt
import cv2
import time
import datetime
import logging

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data import detection_utils as utils
from detectron2.structures import BoxMode

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper   # the default mapper
from detectron2.data import build_detection_train_loader, build_detection_test_loader
from fvcore.transforms.transform import NoOpTransform

from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
import detectron2.utils.comm as comm

## Build custom dataset; register it

In [ ]:
def get_weed_dicts(img_dir, file_list):
    json_file = os.path.join(img_dir, "_weed_labels.json")
    with open(json_file) as f:
        img_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(img_anns.values()):
        if v["filename"] in file_list:  # train or test data
            record = {}
            filename = os.path.join(img_dir, v["filename"])
            # print(filename)
            height, width = cv2.imread(filename).shape[:2]

            record["file_name"] = filename
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width

            annos = v["regions"]   # List of object attributes
            objs = []
            for anno in annos:
                if anno["region_attributes"]["label"] == "weed":
                    sa = anno["shape_attributes"]
                    obj = {
                        "bbox": [sa['x'], sa['y'], sa['width'], sa['height']],
                        "bbox_mode": BoxMode.XYWH_ABS, # or XYXY_ABS
                        "category_id": 0 if anno["region_attributes"]["label"] == "weed" else 1
                    }
                    objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
# Create and register datasets
# img_dir = '/home/mschoder/data/allweeds_600x400/'
img_dir = '/home/mschoder/data/allweeds_1200x800/'

json_file = os.path.join(img_dir, "_weed_labels.json")
print(json_file)
with open(json_file) as f:
    img_anns = json.load(f)

file_list = sorted([k for k in img_anns.keys()])
print("Number of images: ", len(file_list))
np.random.seed(31)
np.random.shuffle(file_list)
val_pct = 0.3  
file_lists = {
    "val": file_list[:int(val_pct * len(file_list))],
    "train": file_list[int(val_pct * len(file_list)):]
}

DatasetCatalog.clear()
for d in ["train", "val"]:
    DatasetCatalog.register("weeds_" + d, lambda d=d: get_weed_dicts(img_dir, file_lists[d]))
    MetadataCatalog.get("weeds_" + d).set(thing_classes=["weed"])
weeds_metadata = MetadataCatalog.get("weeds_train")

# Build full dict
print('Creating datasets...')
# print(file_lists)
dataset_dicts = get_weed_dicts(img_dir, file_lists['train'])
print('Datasets created')

## Build a custom trainer which inherits from default
- Get custom eval metrics while training
- Apply desired augmentations / transformations

In [ ]:
def custom_mapper_train(dataset_dict):
    # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="RGB")
    transform_list = []
    # Rotate 90 degrees to landscape if image is in portrait format
    height, width, _ = image.shape
    if height > width:
        transform_list.append(T.RotationTransform)
    transform_list = [T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                      NoOpTransform(),
                      ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

def custom_mapper_test(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict) 
    image = utils.read_image(dataset_dict["file_name"], format="RGB")
    transform_list = []
    height, width, _ = image.shape
    if height > width:
        transform_list.append(T.RotationTransform)
    transform_list = [NoOpTransform(),
                      ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

## Define hook for validation loss
class ValidationLoss(HookBase):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg.clone()
        self.cfg.DATASETS.TRAIN = cfg.DATASETS.TEST
        self._loader = iter(build_detection_train_loader(self.cfg, mapper=custom_mapper_test)) 
        
    def after_step(self):
        data = next(self._loader)
        with torch.no_grad():
            loss_dict = self.trainer.model(data)
            
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {"val_" + k: v.item() for k, v in 
                                 comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                self.trainer.storage.put_scalars(total_val_loss=losses_reduced, 
                                                 **loss_dict_reduced)

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):

        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"

        return COCOEvaluator(dataset_name, cfg, False, output_folder)

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper_train)
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build_detection_test_loader(cfg, cfg.DATASETS.TEST[0], mapper=custom_mapper_test)


## Define Training Configs

In [ ]:
### TRAINING CONFIG

pretrained_models = ["COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml",
                     "COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml",
                     "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"]

# color_tfs = [NoOpTransform(), HSV_EQ_Transform(), 
#              HLS_EQ_Transform(), NDI_CIVE_ExG_Transform()]
color_tfs = [NoOpTransform()]

m_id = 2
tf_id = 0

# Select base model & initialized weights
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(pretrained_models[m_id]))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(pretrained_models[m_id]) 

cfg.DATASETS.TRAIN = ("weeds_train",)
cfg.DATASETS.TEST = ("weeds_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025  # TODO - tweak
cfg.SOLVER.MAX_ITER = 1000   # Adjusted based on validation mAP (500-1500)

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only one class (weed), other class is background

cfg.TEST.EVAL_PERIOD = 100  # every 5 epochs

cfg.OUTPUT_DIR = '/home/mschoder/experiment_outputs/' + \
                 '1200x800_' + \
                 'model_' + str(pretrained_models[m_id]) + '_' + \
                 'lr_' + str(cfg.SOLVER.BASE_LR) + '_' + \
                 'iters_' + str(750) + '_' + \
                 'tf_' + str(color_tfs[tf_id]) + '/'

# Set everything up to train, register hooks
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
# val_loss = ValidationLoss(cfg)
# trainer.register_hooks([val_loss])
# swap the order of PeriodicWriter and ValidationLoss
# trainer._hooks = trainer._hooks[:-2] + trainer._hooks[-2:][::-1]

trainer.resume_or_load(resume=True)


## Train Model

In [ ]:
# Look at training curves in tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir output 

In [ ]:
# TRAIN
print('Starting training for config: ', cfg.OUTPUT_DIR)
trainer.train()
print('Training succeeded for config: ', cfg.OUTPUT_DIR)

In [ ]:
# Plot Train-Validation Loss curve

# experiment_folder = './output/model_iter4000_lr0005_wf1_date2020_03_20__05_16_45'
# experiment_folder = '/home/mschoder/sugarcane-weed-classification/models/data/output'
# experiment_folder = '/home/mschoder/sugarcane-weed-classification/models/home/mschoder/experiemnt_outputs/model_0_lr_0.00025_iters_1500_tf_NDI_CIVE_ExG_Transform()'
experiment_folder = cfg.OUTPUT_DIR

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(experiment_folder + '/metrics.json')

iters = [x['iteration'] for x in experiment_metrics if 'total_val_loss' in x.keys()]
total_losses = [x['total_loss'] for x in experiment_metrics if 'total_val_loss' in x.keys()]
val_losses = [x['total_val_loss'] for x in experiment_metrics if 'total_val_loss' in x]


# plt.figure(figsize=(10,7))
# plt.plot(iters, total_losses)
# plt.plot(iters, val_losses)
# plt.legend(['total_loss', 'validation_loss'], loc='upper right')
# plt.title('Train-Validation Loss')
# plt.xlabel('Training Iterations')

In [ ]:
### Get mAP validation metrics & others
val_loss_metrics = [x for x in experiment_metrics if 'total_val_loss' in x]
# print(len(val_loss_metrics))

bbox_loss_metrics = [x for x in experiment_metrics if 'bbox/AP50' in x]
# print(len(bbox_loss_metrics))

class_acc_metrics = [x for x in experiment_metrics if 'fast_rcnn/cls_accuracy' in x]
# print(len(class_acc_metrics))

false_neg_metrics = [x for x in experiment_metrics if 'fast_rcnn/false_negative' in x]
# print(len(false_neg_metrics))


In [ ]:
# plt.figure(figsize=(10,7))
# plt.plot([x['iteration'] for x in class_acc_metrics],
#         [x['fast_rcnn/cls_accuracy'] for x in class_acc_metrics])
# plt.plot([x['iteration'] for x in false_neg_metrics],
#         [x['fast_rcnn/false_negative'] for x in false_neg_metrics])
# plt.legend(['class accuracy', 'false_negative'], loc='upper right')
# plt.title('Accuracy and False Negative Rate (Validation)')
# plt.xlabel('Training Iterations')

In [ ]:
val_ap50 = [x['bbox/AP50'] for x in experiment_metrics if 'bbox/AP50' in x]
val_ap_iters = [x['iteration'] for x in experiment_metrics if 'bbox/AP50' in x]
plt.figure(figsize=(10,7))
plt.plot(val_ap_iters, val_ap50)
plt.title('Average Precision at IOU=0.50 (Validation)')
plt.xlabel('Training Iterations')
plt.ylabel('Average Precision')

## Test Results

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
cfg.INSTANCES_CONFIDENCE_THRESH = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode

fig, axs = plt.subplots(2,1, figsize=(12, 15), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .3, wspace=.3)
fig.tight_layout()
axs = axs.ravel()

val_dict = get_weed_dicts(img_dir, file_lists['val'])
for i,d in enumerate(random.sample(val_dict, 2)):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=weeds_metadata, 
                   scale=1)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    axs[i].imshow(out.get_image())


## Processs Video


In [ ]:
def run_video_thru_model(input, output, predictor, out_size=(1200,800), coverage_threshold = None):

    cap = cv2.VideoCapture(input)
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') # Codec format
    out_writer = cv2.VideoWriter(output, fourcc, 20.0, out_size)  

    recent_frames = [0] * 15  # moving average of last N frames

    framecount = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        moving_avg = 0
        if ret==True:
            framecount += 1
            if framecount % 100 == 0:
                print(framecount, " Frames Processed")

            # Get model predictions
            outputs = predictor(frame)
            # print(outputs)

            # Calculate weed coverage
            if coverage_threshold is not None:
                total_area = frame.shape[0] * frame.shape[1]
                weed_area = 0
                for inst in range(len(outputs['instances'])):
                    coords = outputs['instances'][inst].get('pred_boxes').tensor.tolist()[0]
                    box_area = (coords[2] - coords[0]) * (coords[3] - coords[1])
                    weed_area += box_area
                recent_frames = recent_frames[1:]
                recent_frames.append(weed_area/total_area)
                moving_avg = sum(recent_frames) / len(recent_frames)

            # Draw bboxes
            v = Visualizer(frame[:, :, ::-1],
                    metadata=weeds_metadata, 
                    scale=1, 
                    instance_mode=ColorMode.IMAGE
            )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            outframe = out.get_image()[:, :, ::-1]
            outframe = np.array(outframe)

            # Draw activation indicator if above threshold
            if moving_avg > coverage_threshold:
                outframe = cv2.rectangle(outframe,(10,10),(frame.shape[1]-10, frame.shape[0]-10),(0,128,255),12)
            outframe = cv2.putText(outframe, str(round(moving_avg,2)), (285, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2)

            # write the output frame
            out_writer.write(outframe)

            # cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    # Release everything if job is finished
    cap.release()
    out_writer.release()
    cv2.destroyAllWindows()

In [ ]:
##### Uncomment below to run video processing workflow

input_vid = '/home/mschoder/data/raw_video/test_vid_1200x800.avi'
output_vid = '/home/mschoder/data/raw_video/processed_1200x800_hsv_750.avi'
run_video_thru_model(input_vid, output_vid, predictor, coverage_threshold=1.1)